In [1]:
#Imports
import pandas as pd
import csv
from api_keys import census_api
import json
import requests
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter

In [2]:
#Input
traffic_data = "./Resources/US_Accidents_March23.csv"

In [3]:
traffic_df=pd.read_csv(traffic_data)
traffic_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
#Creating DataFrame w/ reduced columns
reduced_traffic_df = traffic_df[["Start_Time","End_Time","Severity","City","State","Weather_Condition",
                                 "Temperature(F)","Visibility(mi)","Wind_Speed(mph)","Precipitation(in)",\
                                 "Sunrise_Sunset","Description","Bump","Crossing", "Give_Way","Junction",\
                                 "No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming",\
                                 "Traffic_Signal","Turning_Loop","Description"
                                ]]
reduced_traffic_df.shape

(7728394, 25)

In [5]:
#Using function to pars
def parse_time(raw):
    date = dt.datetime.fromisoformat(raw).date()
    string = date.isoformat()
    return string

In [6]:
reduced_traffic_df.loc[:, "Start_Date"] = reduced_traffic_df['Start_Time'].map(parse_time)
reduced_traffic_df.loc[:, "End_Date"] = reduced_traffic_df['End_Time'].map(parse_time)
reduced_traffic_df.head()

/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/3816699921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df.loc[:, "Start_Date"] = reduced_traffic_df['Start_Time'].map(parse_time)
/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/3816699921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df.loc[:, "End_Date"] = reduced_traffic_df['End_Time'].map(parse_time)


,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Description,Start_Date,End_Date
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,False,False,False,False,False,False,False,Right lane blocked due to accident on I-70 Eas...,2016-02-08,2016-02-08
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,37.9,10.0,NaN,0.00,...,False,False,False,False,False,False,False,Accident on Brice Rd at Tussing Rd. Expect del...,2016-02-08,2016-02-08
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,36.0,10.0,3.5,NaN,...,False,False,False,False,False,True,False,Accident on OH-32 State Route 32 Westbound at ...,2016-02-08,2016-02-08
3,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,False,False,False,False,False,False,False,Accident on I-75 Southbound at Exits 52 52B US...,2016-02-08,2016-02-08
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,False,False,False,False,False,True,False,Accident on McEwen Rd at OH-725 Miamisburg Cen...,2016-02-08,2016-02-08


In [7]:
def year(raw):
    year_string = raw[:4]
    year_integer = int(year_string)
    return year_integer  

In [8]:
reduced_traffic_df.loc[:, "Start_Year"] = reduced_traffic_df['Start_Date'].map(year)
reduced_traffic_df.loc[:, "End_Year"] = reduced_traffic_df['End_Date'].map(year)
reduced_traffic_df

/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/1789947238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df.loc[:, "Start_Year"] = reduced_traffic_df['Start_Date'].map(year)
/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/1789947238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_traffic_df.loc[:, "End_Year"] = reduced_traffic_df['End_Date'].map(year)


,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Description,Start_Date,End_Date,Start_Year,End_Year
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,False,False,False,False,False,Right lane blocked due to accident on I-70 Eas...,2016-02-08,2016-02-08,2016,2016
1,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,37.9,10.0,NaN,0.00,...,False,False,False,False,False,Accident on Brice Rd at Tussing Rd. Expect del...,2016-02-08,2016-02-08,2016,2016
2,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,36.0,10.0,3.5,NaN,...,False,False,False,True,False,Accident on OH-32 State Route 32 Westbound at ...,2016-02-08,2016-02-08,2016,2016
3,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,False,False,False,False,False,Accident on I-75 Southbound at Exits 52 52B US...,2016-02-08,2016-02-08,2016,2016
4,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,False,False,False,True,False,Accident on McEwen Rd at OH-725 Miamisburg Cen...,2016-02-08,2016-02-08,2016,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,2019-08-23 18:32:01,2,Riverside,CA,Fair,86.0,10.0,13.0,0.00,...,False,False,False,False,False,At Market St - Accident.,2019-08-23,2019-08-23,2019,2019
7728390,2019-08-23 19:11:30,2019-08-23 19:38:23,2,San Diego,CA,Fair,70.0,10.0,6.0,0.00,...,False,False,False,False,False,At Camino Del Rio/Mission Center Rd - Accident.,2019-08-23,2019-08-23,2019,2019
7728391,2019-08-23 19:00:21,2019-08-23 19:28:49,2,Orange,CA,Partly Cloudy,73.0,10.0,10.0,0.00,...,False,False,False,False,False,At Glassell St/Grand Ave - Accident. in the ri...,2019-08-23,2019-08-23,2019,2019
7728392,2019-08-23 19:00:21,2019-08-23 19:29:42,2,Culver City,CA,Fair,71.0,10.0,8.0,0.00,...,False,False,False,False,False,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,2019-08-23,2019-08-23,2019,2019


In [10]:
#dropping NA
drop_traffic_df = reduced_traffic_df.dropna(how='any')
drop_traffic_df.shape

(5382749, 29)

In [11]:
drop_traffic_df.loc[:, "Start_Date"] = drop_traffic_df['Start_Time'].map(parse_time)
drop_traffic_df.loc[:, "End_Date"] = drop_traffic_df['End_Time'].map(parse_time)
drop_traffic_df.head()

,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Description,Start_Date,End_Date,Start_Year,End_Year
5,2016-02-08 07:44:26,2016-02-08 08:14:26,3,Westerville,OH,Light Rain,37.9,7.0,3.5,0.03,...,False,False,False,False,False,Accident on I-270 Outerbelt Northbound near Ex...,2016-02-08,2016-02-08,2016,2016
9,2016-02-08 08:10:04,2016-02-08 08:40:04,3,Westerville,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,False,False,Right hand shoulder blocked due to accident on...,2016-02-08,2016-02-08,2016,2016
11,2016-02-08 08:21:27,2016-02-08 08:51:27,3,Reynoldsburg,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,False,False,One lane blocked due to accident on I-70 Westb...,2016-02-08,2016-02-08,2016,2016
14,2016-02-08 08:39:43,2016-02-08 09:09:43,2,Columbus,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,True,False,Accident on OH-16 Broad St at James Rd. Expect...,2016-02-08,2016-02-08,2016,2016
20,2016-02-08 10:11:15,2016-02-08 10:41:15,2,Columbus,OH,Light Snow,33.8,2.0,4.6,0.01,...,False,False,False,False,False,Accident on Brookhill Dr at Glenhurst Ct.,2016-02-08,2016-02-08,2016,2016


In [12]:
drop_traffic_df.loc[:, "Start_Year"] = drop_traffic_df['Start_Date'].map(year)
drop_traffic_df.loc[:, "End_Year"] = drop_traffic_df['End_Date'].map(year)
drop_traffic_df

,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Description,Start_Date,End_Date,Start_Year,End_Year
5,2016-02-08 07:44:26,2016-02-08 08:14:26,3,Westerville,OH,Light Rain,37.9,7.0,3.5,0.03,...,False,False,False,False,False,Accident on I-270 Outerbelt Northbound near Ex...,2016-02-08,2016-02-08,2016,2016
9,2016-02-08 08:10:04,2016-02-08 08:40:04,3,Westerville,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,False,False,Right hand shoulder blocked due to accident on...,2016-02-08,2016-02-08,2016,2016
11,2016-02-08 08:21:27,2016-02-08 08:51:27,3,Reynoldsburg,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,False,False,One lane blocked due to accident on I-70 Westb...,2016-02-08,2016-02-08,2016,2016
14,2016-02-08 08:39:43,2016-02-08 09:09:43,2,Columbus,OH,Light Rain,37.4,3.0,4.6,0.02,...,False,False,False,True,False,Accident on OH-16 Broad St at James Rd. Expect...,2016-02-08,2016-02-08,2016,2016
20,2016-02-08 10:11:15,2016-02-08 10:41:15,2,Columbus,OH,Light Snow,33.8,2.0,4.6,0.01,...,False,False,False,False,False,Accident on Brookhill Dr at Glenhurst Ct.,2016-02-08,2016-02-08,2016,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,2019-08-23 18:32:01,2,Riverside,CA,Fair,86.0,10.0,13.0,0.00,...,False,False,False,False,False,At Market St - Accident.,2019-08-23,2019-08-23,2019,2019
7728390,2019-08-23 19:11:30,2019-08-23 19:38:23,2,San Diego,CA,Fair,70.0,10.0,6.0,0.00,...,False,False,False,False,False,At Camino Del Rio/Mission Center Rd - Accident.,2019-08-23,2019-08-23,2019,2019
7728391,2019-08-23 19:00:21,2019-08-23 19:28:49,2,Orange,CA,Partly Cloudy,73.0,10.0,10.0,0.00,...,False,False,False,False,False,At Glassell St/Grand Ave - Accident. in the ri...,2019-08-23,2019-08-23,2019,2019
7728392,2019-08-23 19:00:21,2019-08-23 19:29:42,2,Culver City,CA,Fair,71.0,10.0,8.0,0.00,...,False,False,False,False,False,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,2019-08-23,2019-08-23,2019,2019


In [13]:
#reorganizing columns
organized_traffic_df = reduced_traffic_df[["Start_Year","Start_Date","End_Year","End_Date","Start_Time","End_Time",\
                                        "Severity","City","State","Weather_Condition","Temperature(F)","Visibility(mi)",\
                                        "Wind_Speed(mph)","Precipitation(in)","Sunrise_Sunset","Description","Bump",\
                                        "Crossing", "Give_Way","Junction","No_Exit","Railway","Roundabout","Station",\
                                        "Stop","Traffic_Calming","Traffic_Signal","Turning_Loop"]]
organized_traffic_df.head()

,Start_Year,Start_Date,End_Year,End_Date,Start_Time,End_Time,Severity,City,State,Weather_Condition,...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,2016,2016-02-08,2016,2016-02-08,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,...,False,False,False,False,False,False,False,False,False,False
1,2016,2016-02-08,2016,2016-02-08,2016-02-08 06:07:59,2016-02-08 06:37:59,2,Reynoldsburg,OH,Light Rain,...,False,False,False,False,False,False,False,False,False,False
2,2016,2016-02-08,2016,2016-02-08,2016-02-08 06:49:27,2016-02-08 07:19:27,2,Williamsburg,OH,Overcast,...,False,False,False,False,False,False,False,False,True,False
3,2016,2016-02-08,2016,2016-02-08,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,...,False,False,False,False,False,False,False,False,False,False
4,2016,2016-02-08,2016,2016-02-08,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,...,False,False,False,False,False,False,False,False,True,False


In [14]:
organized_traffic_gp = organized_traffic_df.groupby(["Start_Year"]).count()
organized_traffic_gp

,Start_Date,End_Year,End_Date,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
Start_Year,,,,,,,,,,,,,,,,,,,,,
2016,410821,410821,410821,410821,410821,410821,410794,410821,402264,404372,...,410821,410821,410821,410821,410821,410821,410821,410821,410821,410821
2017,718093,718093,718093,718093,718093,718093,718066,718093,700543,703703,...,718093,718093,718093,718093,718093,718093,718093,718093,718093,718093
2018,893426,893426,893426,893426,893426,893426,893416,893426,871904,875415,...,893426,893426,893426,893426,893426,893426,893426,893426,893426,893426
2019,954303,954303,954303,954303,954303,954303,954284,954303,936077,937178,...,954303,954303,954303,954303,954303,954303,954303,954303,954303,954303
2020,1178913,1178913,1178913,1178913,1178913,1178913,1178863,1178913,1148939,1147949,...,1178913,1178913,1178913,1178913,1178913,1178913,1178913,1178913,1178913,1178913
2021,1563753,1563753,1563753,1563753,1563753,1563753,1563700,1563753,1531440,1531731,...,1563753,1563753,1563753,1563753,1563753,1563753,1563753,1563753,1563753,1563753
2022,1762452,1762452,1762452,1762452,1762452,1762452,1762387,1762452,1723764,1723734,...,1762452,1762452,1762452,1762452,1762452,1762452,1762452,1762452,1762452,1762452
2023,246633,246633,246633,246633,246633,246633,246631,246633,240004,240459,...,246633,246633,246633,246633,246633,246633,246633,246633,246633,246633


In [ ]:
x_axis=organized_traffic_gp.index.tolist()
y_axis = organized_traffic_gp["State"].tolist()

plt.ticklabel_format(style='plain')
plt.bar(x_axis, y_axis,color='r',alpha=0.5, align="center")
plt.title("Traffic Accidents YoY")
plt.xlabel("Year")
plt.ylabel("# of Traffic Accidents")

plt.savefig("outputs/AccidentsYoY.png")
plt.show()

In [ ]:
city_traffic = organized_traffic_df.groupby("City")
city_traffic_count = city_traffic.count().sort_values(by=["State"],ascending=False)
city_traffic_count.head(10)
#[city_traffic_count["State"] >=100000]


In [ ]:
day_v_night = organized_traffic_df.groupby(["Start_Year","Sunrise_Sunset"]).count().reset_index()

x_axis = day_v_night["Start_Year"].to_list()
y_axis_1 = day_v_night["Sunrise_Sunset"].tolist()
y_axis_2 = day_v_night["City"].tolist()

plt.plot(x_axis,y_axis_2)

In [15]:
#Calling Census API
base_url = "https://api.census.gov/data/2021/acs/acs5?get=NAME,B01003_001E&for=place:*&in=state:*"
api_url = f"{base_url}&key={census_api}"
json_url = requests.get(api_url).json()
print(api_url)

https://api.census.gov/data/2021/acs/acs5?get=NAME,B01003_001E&for=place:*&in=state:*&key=9adc74e4cfc4d4a300ce521d201d867bcab7e31a


In [16]:
#Creating DataFrame from API
df = pd.DataFrame(json_url)
df

,0,1,2,3
0,NAME,B01003_001E,state,place
1,"Abanda CDP, Alabama",231,01,00100
2,"Abbeville city, Alabama",2231,01,00124
3,"Adamsville city, Alabama",4381,01,00460
4,"Addison town, Alabama",697,01,00484
...,...,...,...,...
31904,"Voladoras comunidad, Puerto Rico",696,72,87638
31905,"Yabucoa zona urbana, Puerto Rico",6466,72,87863
31906,"Yauco zona urbana, Puerto Rico",15404,72,88035
31907,"Yaurel comunidad, Puerto Rico",927,72,88121


In [22]:
#renaming column headers
df.columns=["City","Population","State #","Place #"]
df.head()

,City,Population,State #,Place #
0,NAME,B01003_001E,state,place
1,"Abanda CDP, Alabama",231,01,00100
2,"Abbeville city, Alabama",2231,01,00124
3,"Adamsville city, Alabama",4381,01,00460
4,"Addison town, Alabama",697,01,00484


In [23]:
#drop original column
drop = df.drop([0,0])
drop

,City,Population,State #,Place #
1,"Abanda CDP, Alabama",231,01,00100
2,"Abbeville city, Alabama",2231,01,00124
3,"Adamsville city, Alabama",4381,01,00460
4,"Addison town, Alabama",697,01,00484
5,"Akron town, Alabama",385,01,00676
...,...,...,...,...
31904,"Voladoras comunidad, Puerto Rico",696,72,87638
31905,"Yabucoa zona urbana, Puerto Rico",6466,72,87863
31906,"Yauco zona urbana, Puerto Rico",15404,72,88035
31907,"Yaurel comunidad, Puerto Rico",927,72,88121


In [24]:
#Splitting City and State
split_cl = drop["City"].str.split(',',expand=True)
split_cl

,0,1,2
1,Abanda CDP,Alabama,None
2,Abbeville city,Alabama,None
3,Adamsville city,Alabama,None
4,Addison town,Alabama,None
5,Akron town,Alabama,None
...,...,...,...
31904,Voladoras comunidad,Puerto Rico,None
31905,Yabucoa zona urbana,Puerto Rico,None
31906,Yauco zona urbana,Puerto Rico,None
31907,Yaurel comunidad,Puerto Rico,None


In [25]:
#Renaming Column Header
split_cl.columns=["City_1","State",""]
split_cl

,City_1,State,
1,Abanda CDP,Alabama,None
2,Abbeville city,Alabama,None
3,Adamsville city,Alabama,None
4,Addison town,Alabama,None
5,Akron town,Alabama,None
...,...,...,...
31904,Voladoras comunidad,Puerto Rico,None
31905,Yabucoa zona urbana,Puerto Rico,None
31906,Yauco zona urbana,Puerto Rico,None
31907,Yaurel comunidad,Puerto Rico,None


In [26]:
df =pd.concat([drop,split_cl],axis=1)
drop_df = df.drop([""],axis=1)
drop_df.head()

,City,Population,State #,Place #,City_1,State
1,"Abanda CDP, Alabama",231,01,00100,Abanda CDP,Alabama
2,"Abbeville city, Alabama",2231,01,00124,Abbeville city,Alabama
3,"Adamsville city, Alabama",4381,01,00460,Adamsville city,Alabama
4,"Addison town, Alabama",697,01,00484,Addison town,Alabama
5,"Akron town, Alabama",385,01,00676,Akron town,Alabama


In [27]:
def trim(raw):
    trim = raw.strip()
    return trim

In [28]:
drop_df["State"]= drop_df["State"].map(trim)
drop_df

,City,Population,State #,Place #,City_1,State
1,"Abanda CDP, Alabama",231,01,00100,Abanda CDP,Alabama
2,"Abbeville city, Alabama",2231,01,00124,Abbeville city,Alabama
3,"Adamsville city, Alabama",4381,01,00460,Adamsville city,Alabama
4,"Addison town, Alabama",697,01,00484,Addison town,Alabama
5,"Akron town, Alabama",385,01,00676,Akron town,Alabama
...,...,...,...,...,...,...
31904,"Voladoras comunidad, Puerto Rico",696,72,87638,Voladoras comunidad,Puerto Rico
31905,"Yabucoa zona urbana, Puerto Rico",6466,72,87863,Yabucoa zona urbana,Puerto Rico
31906,"Yauco zona urbana, Puerto Rico",15404,72,88035,Yauco zona urbana,Puerto Rico
31907,"Yaurel comunidad, Puerto Rico",927,72,88121,Yaurel comunidad,Puerto Rico


In [29]:
census_clean_df = drop_df.rename(columns={"City":"City_State","City_1":"City"})
census_clean_df

,City_State,Population,State #,Place #,City,State
1,"Abanda CDP, Alabama",231,01,00100,Abanda CDP,Alabama
2,"Abbeville city, Alabama",2231,01,00124,Abbeville city,Alabama
3,"Adamsville city, Alabama",4381,01,00460,Adamsville city,Alabama
4,"Addison town, Alabama",697,01,00484,Addison town,Alabama
5,"Akron town, Alabama",385,01,00676,Akron town,Alabama
...,...,...,...,...,...,...
31904,"Voladoras comunidad, Puerto Rico",696,72,87638,Voladoras comunidad,Puerto Rico
31905,"Yabucoa zona urbana, Puerto Rico",6466,72,87863,Yabucoa zona urbana,Puerto Rico
31906,"Yauco zona urbana, Puerto Rico",15404,72,88035,Yauco zona urbana,Puerto Rico
31907,"Yaurel comunidad, Puerto Rico",927,72,88121,Yaurel comunidad,Puerto Rico


In [31]:
organized_census_df = census_clean_df[["City","State","Population","State #","Place #","City_State" 
]]
organized_census_df

,City,State,Population,State #,Place #,City_State
1,Abanda CDP,Alabama,231,01,00100,"Abanda CDP, Alabama"
2,Abbeville city,Alabama,2231,01,00124,"Abbeville city, Alabama"
3,Adamsville city,Alabama,4381,01,00460,"Adamsville city, Alabama"
4,Addison town,Alabama,697,01,00484,"Addison town, Alabama"
5,Akron town,Alabama,385,01,00676,"Akron town, Alabama"
...,...,...,...,...,...,...
31904,Voladoras comunidad,Puerto Rico,696,72,87638,"Voladoras comunidad, Puerto Rico"
31905,Yabucoa zona urbana,Puerto Rico,6466,72,87863,"Yabucoa zona urbana, Puerto Rico"
31906,Yauco zona urbana,Puerto Rico,15404,72,88035,"Yauco zona urbana, Puerto Rico"
31907,Yaurel comunidad,Puerto Rico,927,72,88121,"Yaurel comunidad, Puerto Rico"


In [32]:
census_clean_df = organized_census_df[(organized_census_df["City"].str.contains("city")) \
                                      & (organized_census_df["State"] != "Puerto Rico")]
census_clean_df

,City,State,Population,State #,Place #,City_State
2,Abbeville city,Alabama,2231,01,00124,"Abbeville city, Alabama"
3,Adamsville city,Alabama,4381,01,00460,"Adamsville city, Alabama"
6,Alabaster city,Alabama,33133,01,00820,"Alabaster city, Alabama"
7,Albertville city,Alabama,22268,01,00988,"Albertville city, Alabama"
8,Alexander City city,Alabama,14765,01,01132,"Alexander City city, Alabama"
...,...,...,...,...,...,...
31572,Riverton city,Wyoming,10619,56,66220,"Riverton city, Wyoming"
31575,Rock Springs city,Wyoming,23505,56,67235,"Rock Springs city, Wyoming"
31580,Sheridan city,Wyoming,18660,56,69845,"Sheridan city, Wyoming"
31598,Torrington city,Wyoming,6196,56,77530,"Torrington city, Wyoming"


In [33]:
census_clean_df["Population"] = census_clean_df["Population"].astype(int)


/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/2463147141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_clean_df["Population"] = census_clean_df["Population"].astype(int)


In [35]:
def strip(raw):
    city = raw.strip("city")
    trim = city.strip()
    return trim

In [36]:
census_clean_df["City2"] = census_clean_df["City"].map(strip)
census_clean_df

/var/folders/g7/8_nfr5nn6_d576ng21c0x7x40000gn/T/ipykernel_2889/3467919659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_clean_df["City2"] = census_clean_df["City"].map(strip)


,City,State,Population,State #,Place #,City_State,City2
2,Abbeville city,Alabama,2231,01,00124,"Abbeville city, Alabama",Abbeville
3,Adamsville city,Alabama,4381,01,00460,"Adamsville city, Alabama",Adamsville
6,Alabaster city,Alabama,33133,01,00820,"Alabaster city, Alabama",Alabaster
7,Albertville city,Alabama,22268,01,00988,"Albertville city, Alabama",Albertville
8,Alexander City city,Alabama,14765,01,01132,"Alexander City city, Alabama",Alexander City
...,...,...,...,...,...,...,...
31572,Riverton city,Wyoming,10619,56,66220,"Riverton city, Wyoming",Riverton
31575,Rock Springs city,Wyoming,23505,56,67235,"Rock Springs city, Wyoming",Rock Springs
31580,Sheridan city,Wyoming,18660,56,69845,"Sheridan city, Wyoming",Sheridan
31598,Torrington city,Wyoming,6196,56,77530,"Torrington city, Wyoming",Torrington


In [37]:
test1= census_clean_df.rename(columns={"City": "City1","City2": "City"})
test1


,City1,State,Population,State #,Place #,City_State,City
2,Abbeville city,Alabama,2231,01,00124,"Abbeville city, Alabama",Abbeville
3,Adamsville city,Alabama,4381,01,00460,"Adamsville city, Alabama",Adamsville
6,Alabaster city,Alabama,33133,01,00820,"Alabaster city, Alabama",Alabaster
7,Albertville city,Alabama,22268,01,00988,"Albertville city, Alabama",Albertville
8,Alexander City city,Alabama,14765,01,01132,"Alexander City city, Alabama",Alexander City
...,...,...,...,...,...,...,...
31572,Riverton city,Wyoming,10619,56,66220,"Riverton city, Wyoming",Riverton
31575,Rock Springs city,Wyoming,23505,56,67235,"Rock Springs city, Wyoming",Rock Springs
31580,Sheridan city,Wyoming,18660,56,69845,"Sheridan city, Wyoming",Sheridan
31598,Torrington city,Wyoming,6196,56,77530,"Torrington city, Wyoming",Torrington


In [38]:
crosswalk_path = "Resources/crosswalk.csv"
state_abv = pd.read_csv(crosswalk_path)
state_abv

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [40]:
crosswalk_state = pd.merge(test1,state_abv,on="State")

In [41]:
rename_cross = crosswalk_state.rename(columns={"State":"State_old","Abbreviation":"State"})

In [42]:
sorted_og_traffic = organized_traffic_df.sort_values(by=["State","City"],ascending=True)
sorted_og_traffic

,Start_Year,Start_Date,End_Year,End_Date,Start_Time,End_Time,Severity,City,State,Weather_Condition,...,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
523666,2022,2022-08-27,2022,2022-08-27,2022-08-27 14:47:41,2022-08-27 15:17:20,2,Abbeville,AL,Thunder in the Vicinity,...,True,False,False,False,False,False,False,False,False,False
530035,2022,2022-08-20,2022,2022-08-20,2022-08-20 16:32:02,2022-08-20 17:01:44,2,Abbeville,AL,Fair,...,False,False,False,False,False,False,False,False,False,False
536021,2022,2022-08-12,2022,2022-08-12,2022-08-12 20:30:17,2022-08-12 21:19:12,1,Abbeville,AL,Fair,...,True,False,False,False,False,False,False,False,False,False
548951,2022,2022-07-29,2022,2022-07-29,2022-07-29 06:38:49,2022-07-29 07:38:24,2,Abbeville,AL,Mostly Cloudy,...,False,False,False,False,False,False,False,False,False,False
557220,2022,2022-07-20,2022,2022-07-20,2022-07-20 12:57:22,2022-07-20 13:40:00,2,Abbeville,AL,Partly Cloudy,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081190,2021,2021-12-06,2021,2021-12-06,2021-12-06 21:46:08,2021-12-06 23:04:02,4,Wilson,WY,Light Snow,...,False,False,False,False,False,False,False,False,False,False
7529534,2018,2018-05-03,2018,2018-05-03,2018-05-03 08:59:39,2018-05-03 14:59:39,2,Wilson,WY,Scattered Clouds,...,False,False,False,False,False,False,False,False,False,False
3506356,2016,2016-08-01,2016,2016-08-01,2016-08-01 08:41:06,2016-08-01 14:41:06,2,Worland,WY,Clear,...,False,False,False,False,False,False,False,False,False,False
4342352,2023,2023-01-03,2023,2023-01-03,2023-01-03 11:14:06,2023-01-03 12:57:13,2,Yoder,WY,Partly Cloudy,...,False,False,False,True,False,False,False,False,False,False


In [53]:
state_accidents = sorted_og_traffic.groupby(["Start_Year","State","City"]).count()
state_accidents

Start_Date  End_Year  End_Date  Start_Time  \
Start_Year State City                                                       
2016       AL    Ashville               1         1         1           1   
                 Athens                 1         1         1           1   
                 Atmore                 1         1         1           1   
                 Attalla                3         3         3           3   
                 Auburn                 1         1         1           1   
...                                   ...       ...       ...         ...   
2023       WY    Rock Springs          12        12        12          12   
                 Sinclair              36        36        36          36   
                 Wamsutter             77        77        77          77   
                 Wheatland             16        16        16          16   
                 Yoder                  2         2         2           2   

                               End_Time  Severity  Weather_Condition  \
Start_Year State City                                                  
2016       AL    Ashville             1         1                  1   
                 Athens               1         1                  1   
                 Atmore               1         1                  1   
                 Attalla              3         3                  3   
                 Auburn               1         1                  1   
...                                 ...       ...                ...   
2023       WY    Rock Springs        12        12                 12   
                 Sinclair            36        36                 33   
                 Wamsutter           77        77                 29   
                 Wheatland           16        16                 16   
                 Yoder                2         2                  2   

                               Temperature(F)  Visibility(mi)  \
Start_Year State City                                           
2016       AL    Ashville                   1               1   
                 Athens                     1               1   
                 Atmore                     1               1   
                 Attalla                    3               3   
                 Auburn                     1               1   
...                                       ...             ...   
2023       WY    Rock Springs              12              12   
                 Sinclair                  33              33   
                 Wamsutter                 29              29   
                 Wheatland                 16              16   
                 Yoder                      2               2   

                               Wind_Speed(mph)  ...  Give_Way  Junction  \
Start_Year State City                           ...                       
2016       AL    Ashville                    1  ...         1         1   
                 Athens                      1  ...         1         1   
                 Atmore                      1  ...         1         1   
                 Attalla                     0  ...         3         3   
                 Auburn                      1  ...         1         1   
...                                        ...  ...       ...       ...   
2023       WY    Rock Springs               12  ...        12        12   
                 Sinclair                   32  ...        36        36   
                 Wamsutter                  17  ...        77        77   
                 Wheatland                  16  ...        16        16   
                 Yoder                       2  ...         2         2   

                               No_Exit  Railway  Roundabout  Station  Stop  \
Start_Year State City                                                        
2016       AL    Ashville            1        1           1        1     1   
                 Athens             

In [51]:
clean_reset_df = state_accidents.reset_index()

In [52]:
state_city_accident_census = pd.merge(clean_reset_df,rename_cross,on=["State","City"],how="inner")
state_city_accident_census

,Start_Year,State,City,Start_Date,End_Year,End_Date,Start_Time,End_Time,Severity,Weather_Condition,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,City1,State_old,Population,State #,Place #,City_State
0,2016,AL,Ashville,1,1,1,1,1,1,1,...,1,1,1,1,Ashville city,Alabama,2096,01,02908,"Ashville city, Alabama"
1,2018,AL,Ashville,11,11,11,11,11,11,11,...,11,11,11,11,Ashville city,Alabama,2096,01,02908,"Ashville city, Alabama"
2,2019,AL,Ashville,8,8,8,8,8,8,8,...,8,8,8,8,Ashville city,Alabama,2096,01,02908,"Ashville city, Alabama"
3,2020,AL,Ashville,7,7,7,7,7,7,7,...,7,7,7,7,Ashville city,Alabama,2096,01,02908,"Ashville city, Alabama"
4,2021,AL,Ashville,18,18,18,18,18,18,15,...,18,18,18,18,Ashville city,Alabama,2096,01,02908,"Ashville city, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36919,2023,TX,Talco,1,1,1,1,1,1,1,...,1,1,1,1,Talco city,Texas,577,48,71732,"Talco city, Texas"
36920,2023,TX,Trenton,3,3,3,3,3,3,3,...,3,3,3,3,Trenton city,Texas,729,48,73592,"Trenton city, Texas"
36921,2023,WV,Follansbee,1,1,1,1,1,1,1,...,1,1,1,1,Follansbee city,West Virginia,2842,54,28204,"Follansbee city, West Virginia"
36922,2023,WV,McMechen,1,1,1,1,1,1,1,...,1,1,1,1,McMechen city,West Virginia,1667,54,50260,"McMechen city, West Virginia"


In [56]:
city_accident_census = state_city_accident_census[["State","City","Start_Year","Population","Start_Date"]]
city_accident_census

,State,City,Start_Year,Population,Start_Date
0,AL,Ashville,2016,2096,1
1,AL,Ashville,2018,2096,11
2,AL,Ashville,2019,2096,8
3,AL,Ashville,2020,2096,7
4,AL,Ashville,2021,2096,18
...,...,...,...,...,...
36919,TX,Talco,2023,577,1
36920,TX,Trenton,2023,729,3
36921,WV,Follansbee,2023,2842,1
36922,WV,McMechen,2023,1667,1


In [55]:
census_traffic_df = pd.merge(reduced_traffic_df,rename_cross,on=["State","City"],how="inner")
census_traffic_df

,Start_Time,End_Time,Severity,City,State,Weather_Condition,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Start_Date,End_Date,Start_Year,End_Year,City1,State_old,Population,State #,Place #,City_State
0,2016-02-08 05:46:00,2016-02-08 11:00:00,3,Dayton,OH,Light Rain,36.9,10.0,NaN,0.02,...,2016-02-08,2016-02-08,2016,2016,Dayton city,Ohio,138416,39,21000,"Dayton city, Ohio"
1,2016-02-08 07:23:34,2016-02-08 07:53:34,3,Dayton,OH,Mostly Cloudy,35.1,9.0,4.6,NaN,...,2016-02-08,2016-02-08,2016,2016,Dayton city,Ohio,138416,39,21000,"Dayton city, Ohio"
2,2016-02-08 07:39:07,2016-02-08 08:09:07,2,Dayton,OH,Mostly Cloudy,36.0,6.0,3.5,NaN,...,2016-02-08,2016-02-08,2016,2016,Dayton city,Ohio,138416,39,21000,"Dayton city, Ohio"
3,2016-02-08 07:59:35,2016-02-08 08:29:35,2,Dayton,OH,Overcast,34.0,7.0,3.5,NaN,...,2016-02-08,2016-02-08,2016,2016,Dayton city,Ohio,138416,39,21000,"Dayton city, Ohio"
4,2016-02-08 07:59:58,2016-02-08 08:29:58,3,Dayton,OH,Overcast,34.0,7.0,3.5,NaN,...,2016-02-08,2016-02-08,2016,2016,Dayton city,Ohio,138416,39,21000,"Dayton city, Ohio"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545883,2019-08-02 17:05:08,2019-08-02 17:33:21,4,Ness City,KS,Mostly Cloudy,85.0,10.0,5.0,0.00,...,2019-08-02,2019-08-02,2019,2019,Ness City city,Kansas,1301,20,49925,"Ness City city, Kansas"
5545884,2019-08-04 01:38:48,2019-08-04 02:06:21,4,Russell,IA,Fair,68.0,10.0,3.0,0.00,...,2019-08-04,2019-08-04,2019,2019,Russell city,Iowa,698,19,69285,"Russell city, Iowa"
5545885,2019-08-05 19:32:48,2019-08-05 19:59:59,4,Meadville,MO,NaN,82.0,NaN,6.0,0.00,...,2019-08-05,2019-08-05,2019,2019,Meadville city,Missouri,525,29,47036,"Meadville city, Missouri"
5545886,2019-08-06 16:37:31,2019-08-06 17:05:28,4,Greenfield,IA,Fair,82.0,10.0,5.0,0.00,...,2019-08-06,2019-08-06,2019,2019,Greenfield city,Iowa,1671,19,32790,"Greenfield city, Iowa"


In [57]:
census_accidents_pop =city_accident_census.rename(columns={"Start_Date":"Accidents"})
census_accidents_pop

,State,City,Start_Year,Population,Accidents
0,AL,Ashville,2016,2096,1
1,AL,Ashville,2018,2096,11
2,AL,Ashville,2019,2096,8
3,AL,Ashville,2020,2096,7
4,AL,Ashville,2021,2096,18
...,...,...,...,...,...
36919,TX,Talco,2023,577,1
36920,TX,Trenton,2023,729,3
36921,WV,Follansbee,2023,2842,1
36922,WV,McMechen,2023,1667,1


In [58]:
census_accidents_pop["pop_per_100K"] = census_accidents_pop["Population"]/100000
census_accidents_pop

,State,City,Start_Year,Population,Accidents,pop_per_100K
0,AL,Ashville,2016,2096,1,0.02096
1,AL,Ashville,2018,2096,11,0.02096
2,AL,Ashville,2019,2096,8,0.02096
3,AL,Ashville,2020,2096,7,0.02096
4,AL,Ashville,2021,2096,18,0.02096
...,...,...,...,...,...,...
36919,TX,Talco,2023,577,1,0.00577
36920,TX,Trenton,2023,729,3,0.00729
36921,WV,Follansbee,2023,2842,1,0.02842
36922,WV,McMechen,2023,1667,1,0.01667


In [59]:
census_accidents_pop["accidents_per_pop_100K"] = census_accidents_pop["Accidents"] / census_accidents_pop["pop_per_100K"]
census_accidents_pop

,State,City,Start_Year,Population,Accidents,pop_per_100K,accidents_per_pop_100K
0,AL,Ashville,2016,2096,1,0.02096,47.709924
1,AL,Ashville,2018,2096,11,0.02096,524.809160
2,AL,Ashville,2019,2096,8,0.02096,381.679389
3,AL,Ashville,2020,2096,7,0.02096,333.969466
4,AL,Ashville,2021,2096,18,0.02096,858.778626
...,...,...,...,...,...,...,...
36919,TX,Talco,2023,577,1,0.00577,173.310225
36920,TX,Trenton,2023,729,3,0.00729,411.522634
36921,WV,Follansbee,2023,2842,1,0.02842,35.186488
36922,WV,McMechen,2023,1667,1,0.01667,59.988002


In [60]:
major_cities = census_accidents_pop[census_accidents_pop["Population"]>=100000]

In [61]:
major_cities.sort_values(by=["accidents_per_pop_100K"],ascending=False)

,State,City,Start_Year,Population,Accidents,pop_per_100K,accidents_per_pop_100K
5112,FL,Miami,2022,440807,64505,4.40807,14633.388308
5111,FL,Miami,2021,440807,58816,4.40807,13342.800818
5317,FL,Orlando,2022,302968,36402,3.02968,12015.130311
5316,FL,Orlando,2021,302968,33014,3.02968,10896.860394
11119,LA,Baton Rouge,2021,225539,14552,2.25539,6452.099193
...,...,...,...,...,...,...,...
20377,TX,Amarillo,2016,200371,1,2.00371,0.499074
264,AR,Little Rock,2016,201893,1,2.01893,0.495312
31833,FL,Port St. Lucie,2018,202177,1,2.02177,0.494616
30516,TX,Lubbock,2023,255537,1,2.55537,0.391333


In [63]:
accidents_per_pop = census_accidents_pop["Accidents"]/census_accidents_pop["Population"]
accidents_per_pop

0        0.000477
1        0.005248
2        0.003817
3        0.003340
4        0.008588
           ...   
36919    0.001733
36920    0.004115
36921    0.000352
36922    0.000600
36923    0.000244
Length: 36924, dtype: float64

In [68]:
accident_by_city = major_cities.groupby(["Start_Year","State","City"]).count().sort_values(by=["accidents_per_pop_100K"],ascending=False)
accident_by_city

Population  Accidents  pop_per_100K  \
Start_Year State City                                                  
2016       AL    Birmingham               1          1             1   
2021       CA    San Bernardino           1          1             1   
                 West Covina              1          1             1   
           CO    Arvada                   1          1             1   
                 Aurora                   1          1             1   
...                                     ...        ...           ...   
2018       NM    Las Cruces               1          1             1   
                 Rio Rancho               1          1             1   
           NV    Reno                     1          1             1   
                 Sparks                   1          1             1   
2023       WI    Milwaukee                1          1             1   

                                 accidents_per_pop_100K  
Start_Year State City                                    
2016       AL    Birmingham                           1  
2021       CA    San Bernardino                       1  
                 West Covina                          1  
           CO    Arvada                               1  
                 Aurora                               1  
...                                                 ...  
2018       NM    Las Cruces                           1  
                 Rio Rancho                           1  
           NV    Reno                                 1  
                 Sparks                               1  
2023       WI    Milwaukee                            1  

[2327 rows x 4 columns]